In [ ]:
## importing stuff
import os
import pickle
from sys import path as syspath
syspath.append(os.path.expanduser("~/srdjan_functs/"))
import javabridge
from bioformats import JARS as bfJARS
javabridge.start_vm(class_path=bfJARS, max_heap_size="20G")
import numpy as np
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2
# %aimport islets

from IPython.display import display
import pandas as pd

from islets.Recording import Recording, saveMovie
from islets.Regions import Regions
from islets.numeric import power_spectrum, rebin
from islets.utils import show_movie, saveRois
from islets.Regions import getPeak2BoundaryDF, getGraph_of_ROIs_to_Merge, mergeBasedOnGraph

from caiman import movie as cmovie

from matplotlib.colors import LogNorm

In [ ]:
plt.rcParams["animation.embed_limit"] = 200

In [ ]:
# recFile = "/data/Sandra/2019/2019_10_07/Experiment32b.lif" 
recFile = "/data/Sandra/2019/2019_10_08/Experiment33a.lif" 

In [ ]:
rec = Recording(recFile)

In [ ]:
rec.metadata

In [ ]:
# rec.parse_metadata()

In [ ]:
iSeries = 7
metadata = rec.metadata.loc[iSeries]

In [ ]:
rec.metadata["Frequency"] = 8.4

In [ ]:
rec.import_series("Series023")

In [ ]:
data = np.array([[rec.rdr.read(series=iSeries,t=t,z=z,c=0, rescale=False) for z in range(metadata.SizeZ)] for t in range(metadata.SizeT)])

In [ ]:
data.max()

In [ ]:
data.shape

In [ ]:
mean3D = data.mean(axis=0)
# std3D = data.std(axis=0)
# diff_std3D = np.diff(data).std(axis=0)

In [ ]:
show_movie(mean3D,log=True, autoadjust=False, figScale=3)

In [ ]:
import plotly_express as plx

In [ ]:
# s3d = mean3D[:,::,::]
# z,x,y = np.meshgrid(*map(range,s3d.shape), indexing="ij")
# fltr = s3d>np.percentile(s3d,99)
# x = x[fltr]
# y = y[fltr]
# z = z[fltr]
# plx.scatter_3d(
#     x=x,
#     y=y,
#     z=z,
# #     size=np.clip(s3d[fltr]/100,.0000001,.1),
#     size=s3d[fltr]**2/100,
# #     size=[.001]*fltr.sum()
# )
# # plx.scatter_3d(
# #     x=x.flat,
# #     y=y.flat,
# #     z=z.flat,
# #     size=(s3d**4/1000).flatten(),
# # )

In [ ]:
# rrs = [Regions(cmovie(data[:,z], fr=metadata.Frequency), gSig_filt=(3,4), diag=True) for z in range(data.shape[1])]

In [ ]:
# for r in rrs:
#     r.calcTraces()

In [ ]:
# rrs[-4].examine()

In [ ]:
from islets.Regions3D import climb

In [ ]:
from islets.Regions3D import crawlDict

In [ ]:
from cv2 import GaussianBlur

In [ ]:
mean3D_blurred = GaussianBlur(mean3D.T,(5,5),-1).T
mean3D_blurred = GaussianBlur(mean3D_blurred,(3,1),-1)

In [ ]:
b = crawlDict(mean3D_blurred[:], processes=10)
len(b)

In [ ]:
from islets.Regions3D import Regions as Reg3D

In [ ]:
r3d = Reg3D(b, full=False)

In [ ]:
from collections import OrderedDict

In [ ]:
r3d.df = pd.DataFrame(OrderedDict([
            ("peak",  list(b.keys())),
            ("pixels",list(b.values())),
            ("size",  list(map(len, b.values())))
        ]))

In [ ]:
r3d.statImages = {"mean":mean3D}

In [ ]:
plt.plot(np.log10(r3d.df["size"].sort_values()), np.arange(len(r3d.df)))

In [ ]:
r3d.df.drop(index= r3d.df.query("size<100").index, inplace=True)

In [ ]:
r3d.df.shape

In [ ]:
r3d.calcTraces(data, fr=metadata.Frequency)

In [ ]:
r3d.df.columns

In [ ]:
r3d.Freq = 8.4

In [ ]:
r3d.time = np.arange(len(data))/8.4

In [ ]:
ts=5
r3d.fast_filter_traces(ts,Npoints=np.inf, z_sp=3)

In [ ]:
zz = np.vstack(r3d.df["zScore_%g"%ts])

In [ ]:
df = r3d.df.query("size>500")
z,x,y = np.vstack(df.peak).T

fig = plx.scatter_3d(
    x=x*metadata.pxSize,
    y=y*metadata.pxSize,
    z=z*1.93,
    size=df["size"],
    size_max = 30,
#     color=np.log(10+np.sum(zz>3,1))
    color=np.mean(zz[r3d.df.index.isin(df.index),:len(r3d.time)//2]>3,1),
#     hoverinfo="text",
#     hovertext=r3d.df.index
#     size=[.001]*fltr.sum()
)
fig.update_layout(scene_aspectmode='data')

In [ ]:
df = r3d.df.query("size>500")
z,x,y = np.vstack(df.peak).T

fig = plx.scatter_3d(
    x=x*metadata.pxSize,
    y=y*metadata.pxSize,
    z=z*1.93,
    size=df["size"],
    size_max = 30,
#     color=np.log(10+np.sum(zz>3,1))
    color=np.mean(zz[r3d.df.index.isin(df.index),len(r3d.time)//2:]>3,1),
#     hoverinfo="text",
#     hovertext=r3d.df.index
#     size=[.001]*fltr.sum()
)
fig.update_layout(scene_aspectmode='data')

In [ ]:
%matplotlib inline

In [ ]:
plt.plot(
    (zz[:,:zz.shape[1]//2]>3).mean(1),
    (zz[:,zz.shape[1]//2:]>3).mean(1),"."
)
plt.xscale("log")
plt.yscale("log")
plt.plot([1e-3,1],[1e-3,1])
plt.gca().set_aspect("equal")

In [ ]:
%matplotlib notebook

In [ ]:
try:
    t = r3d.showTime['%g'%ts]
except:
    t = r3d.time
plt.figure(figsize=(10,4))
ia = 0
for ix in r3d.df.index[np.argsort(np.sum(zz>4,axis=1))[::-1]][:10]:
    plt.plot(t, r3d.df.loc[ix,"zScore_%g"%ts]+20*ia,lw=.5)
    ia +=1 

In [ ]:
plt.plot(zz.mean(1), r3d.df["size"],".")
plt.xscale("log")
plt.yscale("log")